In [1]:
#imports 
import numpy as np
import splat
import pandas as pd
import glob
from astropy.io import ascii
import matplotlib.pyplot as plt
from tqdm import tqdm
import seaborn as sns
import popsims
#splat.initializeStandards(all=True)
from scipy.interpolate import interp1d
import splat.empirical as spe

from popsims.relations import polynomial_relation
from popsims.filtermags import filterMag
%matplotlib inline


ModuleNotFoundError: No module named 'splat'

In [ ]:
#splat.initializeStandards()

In [ ]:
#functions
def k_clip_fit(x, y, sigma_y, sigma = 5, n=6):
    
    '''Fit a polynomial to y vs. x, and k-sigma clip until convergence
    hard-coded, returns mask array
    '''
    
    not_clipped = np.ones_like(y).astype(bool)
    n_remove = 1
    
    while n_remove > 0:

        best_fit = np.poly1d(np.polyfit(x[not_clipped], y[not_clipped], n))
        
        norm_res = (np.abs(y - best_fit(x)))/(sigma_y)
        remove = np.logical_and(norm_res >= sigma, not_clipped == 1)
        n_remove = sum(remove)
        not_clipped[remove] = 0   
        
    return  not_clipped

def fit_with_nsigma_clipping(x, y, y_unc, n, sigma=4.):
    not_clipped = k_clip_fit(x, y, y_unc, sigma = sigma)
    return not_clipped, np.poly1d(np.polyfit(x[not_clipped], y[not_clipped], n))

def compute_mags_from_reference(spt, mag_key, ref):
    vals, unc= polynomial_relation(spt, 'spt', mag_key, ref, nsample=1000, xerr=0.0)
    return vals, unc

def get_correct_filter_name(flt):
    #reformat from by 
    f= flt.split('_')
    if f[-1]=='ps1':
        return 'PANSTARRS_'+(f[0]).upper()
    else:
        return  flt
    
def get_best2018_relation(spt, flt):
    return spe.typeToMag(spt, flt, ref='best2018')

def get_colors(sp, flt1, flt2):
    #measuring filtermags in for two filters and comparing that to target filters
    mag, mag_unc = filterMag(sp.wave, sp.flux, sp.noise, sp.flux.unit, flt1)
    #calculate the mag of the standard in J and H
    mag2, mag2_unc = filterMag(sp.wave, sp.flux, sp.noise, sp.flux.unit, flt2)
    #calculate the offset 
    return mag2-mag, (mag2_unc**2+ mag_unc**2)**0.5


def compute_colors(sp, sd=False):
    fl_pairs=None
    if sd: fl_pairs=sd_pairs
    if not sd: fl_pairs=dwarf_pairs

    res={}
    for p in fl_pairs.keys():
        flt=get_correct_filter_name(fl_pairs[p]['ref'])
        res.update({p+'and'+flt: \
                    get_colors(sp, p,flt)})
    return res

def spt_to_teff(spt, sd=True):
    if sd:
        if spt >25:   #use dwarfs for later than L5
            return  popsims.relations.spt_to_teff_pecaut(spt)
        else:
            return spe.typeToTeff(spt, reverse=False, ref='gonzales18')[0].value
    if not sd:
        return popsims.relations.spt_to_teff_pecaut(spt)
    
def teff_to_spt(teff, sd=True):
    if sd:
        if teff <1600: #use dwarfs for higher teff 
            return popsims.relations.teff_to_spt_pecaut(teff).flatten()[0]
        else:
            return spe.typeToTeff(teff, reverse=True, ref='gonzales18')[0]
    if not sd:
        return popsims.relations.teff_to_spt_pecaut(teff)

In [ ]:
popsims.relations.spt_to_teff_pecaut(25)

In [ ]:
for f in glob.glob(popsims.filtermags.FILTER_FOLDER+'/*.dat'):
    #transform to .txt
    try:
        d=pd.read_csv(f, names=['LAM', 'TRANSMISSION'], sep=' ').astype(float)
        d['LAM']= d['LAM']/10_000
        d.to_csv(f.replace('.dat', '.txt'), sep=' ', index=False)
        plt.plot(d.LAM, d.TRANSMISSION)
    except ValueError:
        pass

In [ ]:
#dwarfs 
#first look at the previous absolute mag relations for optical
dwarf_pairs={'LSST_G': {'ref': 'g_ps1', 'method':'vega'},
            'LSST_R': {'ref': 'r_ps1', 'method':'vega'},
            'LSST_I': {'ref': 'i_ps1', 'method': 'vega'},
            'LSST_Y': {'ref': 'y_ps1', 'method': 'vega'},
            'LSST_Z':{'ref': 'z_ps1', 'method': 'vega'},
             
             #ADD SLOAN WHILE YOU'RE AT IT
            'SDSS_G': {'ref': 'g_ps1', 'method':'vega'},
            'SDSS_R': {'ref': 'r_ps1', 'method':'vega'},
            'SDSS_I': {'ref': 'i_ps1', 'method': 'vega'},
            'SDSS_Z':{'ref': 'z_ps1', 'method': 'vega'},
             
             #VISTA
            'VISTA_Z':{'ref': 'z_ps1', 'method': 'vega'},
            'VISTA_Y':{'ref': 'y_ps1', 'method': 'vega'},
           'VISTA_J': {'ref': '2MASS J', 'method': 'vega'},
            'VISTA_H': {'ref': '2MASS H', 'method': 'vega'},
             'VISTA_KS':{'ref': '2MASS K', 'method': 'vega'},
             
             #UKIDSS
            'UKIDSS_Y': {'ref': 'y_ps1', 'method': 'vega'},
            'UKIDSS_Z': {'ref': 'z_ps1', 'method': 'vega'},
            'UKIDSS_J': {'ref': '2MASS J', 'method': 'vega'},
            'UKIDSS_H': {'ref': '2MASS H' , 'method': 'vega'},
             'UKIDSS_K':{'ref': '2MASS K', 'method': 'vega'},
             
             #JWST
             'NIRISS_F090W': {'ref': 'r_ps1', 'method': 'vega'},
             'NIRISS_F115W': {'ref': '2MASS J', 'method': 'vega'},
             'NIRISS_F150W': {'ref': '2MASS J', 'method': 'vega'},
             'NIRISS_F200W': {'ref': '2MASS H', 'method': 'vega'},
             'NIRISS_F200W': {'ref': '2MASS H', 'method': 'vega'},
             
             #ROMAN
             'WFI_R062': {'ref': 'z_ps1', 'method': 'vega'},
             'WFI_Z087': {'ref': 'z_ps1', 'method': 'vega'},
             'WFI_Y106': {'ref': 'y_ps1', 'method': 'vega'},
             'WFI_J129': {'ref': '2MASS J', 'method': 'vega'},
             'WFI_H158': {'ref': '2MASS H', 'method': 'vega'},
             'WFI_F184': {'ref': '2MASS H', 'method': 'vega'},
             'WFI_Prism': {'ref': '2MASS J', 'method': 'vega'},
             'WFI_Grism': {'ref': '2MASS H', 'method': 'vega'},
             
             #EUCLID
             'EUCLID_Y': {'ref': 'y_ps1', 'method': 'vega'},
             'EUCLID_J': {'ref': '2MASS J', 'method': 'vega'},
             'EUCLID_H': {'ref': '2MASS H', 'method': 'vega'}}


In [ ]:
spgrid=np.arange(10, 40)

In [ ]:
#GET TEMPLATES=
dwarf_tpls= np.load('/Volumes/LaCie/popsimsdata/dwarfs_templates.npy',  allow_pickle=True)[0]
FITS= np.load('/Volumes/LaCie/popsimsdata//abs_mag_relations.npy', allow_pickle=True)[0]

In [ ]:
len([x.name for x in dwarf_tpls['spectra'] if 'Aurora' in x.name])

In [ ]:
splat.searchLibrary().columns

In [ ]:
get_correct_filter_name('2MASS J')

In [ ]:
#compute color corrections
color_corrections=[ compute_colors(sp, sd=False) for sp in tqdm(dwarf_tpls['spectra'])]

In [ ]:
df_dwarfs=pd.DataFrame.from_records(color_corrections)
df_dwarfs['spt']= dwarf_tpls['spt']
df_dwarfs['teff']= [ spt_to_teff(x, sd=False) for x in df_dwarfs.spt]

In [ ]:
from scipy.interpolate import UnivariateSpline

In [ ]:
#fit to color corrections
cols=[k for k in df_dwarfs.keys() if k not in ['spt', 'teff']]
for k in cols:
        fig, ax=plt.subplots()

        mask= ~np.logical_or.reduce([np.isnan(df_dwarfs.spt.values), 
                                    np.isnan( np.vstack(df_dwarfs[k].values)[:,0]),
                                    np.isnan(np.vstack(df_dwarfs[k].values)[:,1]),
                                    np.vstack(df_dwarfs[k].values)[:,1] > 2e-1])
         #FIT
        x=df_dwarfs.spt.values.astype(float)[mask]
        y= np.vstack(df_dwarfs[k].values)[:,0][mask]
        yerr= np.vstack(df_dwarfs[k].values)[:,1][mask]
        #mask models that are nans

        m, fit= fit_with_nsigma_clipping(x, y, yerr, 3, sigma=1.)
        
        try:

            fit= UnivariateSpline(x[np.argsort(x)], y[np.argsort(x)])
        except:
            pass




        FITS['colors_spt']['dwarfs'].update({k: {'x': 'spt',
                                              'y': k,
                                              'fit':fit, \
                                              'scatter': np.nanmedian(yerr), \
                                              'method': 'spline',\
                                              'range':  (np.nanmin(x),\
                                                         np.nanmax(x)), \
                                             'x0':0}})

        xlim=(np.nanmin(x),\
                                                         np.nanmax(x))
        xgrid=np.arange(xlim[0], xlim[1])

        ylim=fit(list(xlim))

        ylim=[ylim[0]-5*np.nanmedian(yerr), ylim[-1]+5*np.nanmedian(yerr)]
        xlim=[xlim[0]-1, xlim[1]+1]

        ax.errorbar(x, y, yerr=yerr, fmt='o', alpha=0.2)
        ax.plot(xgrid, fit(xgrid) )

        ax.set(xlabel='Spectral Type', ylabel=k.replace('and', '-'), ylim=[-1, 1])

In [ ]:
#use freezer et al for >16
gs= get_best2018_relation(dwarf_tpls['spt'], 'PANSTARRS_G')
rs=compute_mags_from_reference(dwarf_tpls['spt'], 'r_ps1', 'freeser2022')
imags=compute_mags_from_reference(dwarf_tpls['spt'], 'i_ps1', 'freeser2022')
zs=compute_mags_from_reference(dwarf_tpls['spt'], 'z_ps1', 'freeser2022')
ys=compute_mags_from_reference(dwarf_tpls['spt'], 'y_ps1', 'freeser2022')

#use beset et al for <16
best_gs= get_best2018_relation(dwarf_tpls['spt'], 'PANSTARRS_G')
best_rs= get_best2018_relation(dwarf_tpls['spt'], 'PANSTARRS_R')
best_is= get_best2018_relation(dwarf_tpls['spt'], 'PANSTARRS_I')
best_zs= get_best2018_relation(dwarf_tpls['spt'], 'PANSTARRS_Z')
best_ys= get_best2018_relation(dwarf_tpls['spt'], 'PANSTARRS_Y')

#mask
mask= df_dwarfs['spt']<=20

In [ ]:
df_dwarfs['PANSTARRS_G']=gs[0]
df_dwarfs['PANSTARRS_G_er']=gs[1]
df_dwarfs['PANSTARRS_R']=rs[0]
df_dwarfs['PANSTARRS_R_er']=rs[1]
df_dwarfs['PANSTARRS_I']=imags[0]
df_dwarfs['PANSTARRS_I_er']=imags[1]
df_dwarfs['PANSTARRS_Z']=zs[0]
df_dwarfs['PANSTARRS_Z_er']=zs[1]
df_dwarfs['PANSTARRS_Y']=ys[0]
df_dwarfs['PANSTARRS_Y_er']=ys[1]

df_dwarfs['PANSTARRS_G'][mask]=best_gs[0][mask]
df_dwarfs['PANSTARRS_G_er'][mask]=best_gs[1][mask]
df_dwarfs['PANSTARRS_R'][mask]=best_rs[0][mask]
df_dwarfs['PANSTARRS_R_er'][mask]=best_rs[1][mask]
df_dwarfs['PANSTARRS_I'][mask]=best_is[0][mask]
df_dwarfs['PANSTARRS_I_er'][mask]=best_is[1][mask]
df_dwarfs['PANSTARRS_Z'][mask]=best_zs[0][mask]
df_dwarfs['PANSTARRS_Z_er'][mask]=best_zs[1][mask]
df_dwarfs['PANSTARRS_Y'][mask]=best_ys[0][mask]
df_dwarfs['PANSTARRS_Y_er'][mask]=best_ys[1][mask]

#add uncertainties

In [ ]:
def get_j_mag(spt):
    if spt <=20:
        return popsims.relations.absolute_mag_from_spt(spt, mag='j', ref='dupuy2012', syst='2mass')
    else:
        return popsims.relations.absolute_mag_from_spt(spt, mag='j', ref='dupuy2012', syst='2mass')
    
def get_h_mag(spt):
    if spt <=20:
        return popsims.relations.absolute_mag_from_spt(spt, mag='h', ref='dupuy2012', syst='2mass')
    else:
        return popsims.relations.absolute_mag_from_spt(spt, mag='h', ref='dupuy2012', syst='2mass')
        
def get_k_mag(spt):
    if spt <=20:
        return  popsims.relations.absolute_mag_from_spt(spt, mag='ks', ref='dupuy2012', syst='2mass')
    else:
        return  popsims.relations.absolute_mag_from_spt(spt, mag='k', ref='freeser2022', syst='2mass')

In [ ]:
get_j_mag(20)

In [ ]:
js=np.vstack( [ get_j_mag(x) for x in df_dwarfs.spt.values])
hs=np.vstack( [ get_h_mag(x) for x in df_dwarfs.spt.values])
ks=np.vstack( [ get_k_mag(x) for x in df_dwarfs.spt.values])
df_dwarfs['2MASS J']= js[:,0]
df_dwarfs['2MASS H']= hs[:,0]
df_dwarfs['2MASS K']= ks[:,0]
df_dwarfs['2MASS J_er']= js[:,1]
df_dwarfs['2MASS H_er']= hs[:,1]
df_dwarfs['2MASS K_er']= ks[:,1]

In [ ]:
fig, ax=plt.subplots()
ax.errorbar(df_dwarfs.spt.values, js[:,0], yerr=js[:,1], fmt='o')
ax.set(xlabel='Spectral Type', ylabel='J 2MASS')

fig, ax=plt.subplots()
ax.errorbar(df_dwarfs.spt.values, hs[:,0], yerr=hs[:,1], fmt='o')
ax.set(xlabel='Spectral Type', ylabel='H 2MASS')


fig, ax=plt.subplots()
ax.errorbar(df_dwarfs.spt.values, ks[:,0], yerr=ks[:,1], fmt='o')
ax.set(xlabel='Spectral Type', ylabel='K 2MASS')

In [ ]:
#popsims.relations.absolute_mag_from_spt(30, mag='k', ref='freeser2022', syst='2mass')

In [ ]:
#compute absolute magnitudes
for k in dwarf_pairs.keys():
    #compute corr
    ref_k= get_correct_filter_name(dwarf_pairs[k]['ref'])
    corrk=   k+'and'+ref_k
    mags= df_dwarfs[ref_k]+np.vstack(df_dwarfs[corrk])[:,0]
    unc= (df_dwarfs[ref_k+'_er']**2+np.vstack(df_dwarfs[corrk])[:,1]**2)**0.5
    
    df_dwarfs[k]= mags
    df_dwarfs[k+'_er']=unc
    print (k, np.nanmedian(unc))
    

In [ ]:
line_models=np.load('/Volumes/LaCie/popsimsdata/subdwarfs_models_line2021.npy', allow_pickle=True)[0]
line_models_bool=line_models['z']==0.0

In [ ]:
model_vals=dict(zip(dwarf_pairs.keys(), []))

#compute values from filtermags
for k in dwarf_pairs.keys():
    mags= np.vstack([filterMag(x.wave, x.flux, x.noise, x.flux_unit, k) for x in np.array(line_models['spectra'])[line_models_bool]])
    
    model_vals[k]=mags[:,0] 
    model_vals[k+'_er']=np.nanmedian(df_dwarfs[k+'_er'][df_dwarfs[k+'_er']<1.])
    
model_line_df=pd.DataFrame(model_vals)

model_line_df['spt']=[ teff_to_spt(x, sd=False) for x in np.array(line_models['teff'])[line_models_bool]]
model_line_df['teff']= np.array(line_models['teff'])[line_models_bool]
model_line_df=model_line_df.applymap(lambda x: float(x))


In [ ]:
#ADD models
models=np.load('/Volumes/LaCie/popsimsdata/dwarfs_models.npy', allow_pickle=True)[0]
model_vals=dict(zip(dwarf_pairs.keys(), []))

#compute values from filtermags
for k in dwarf_pairs.keys():
    mags= np.vstack([filterMag(x.wave, x.flux, x.noise, x.flux_unit, k) for x in models['spectra']])
    
    model_vals[k]=mags[:,0] 
    model_vals[k+'_er']=np.nanmedian(df_dwarfs[k+'_er'][df_dwarfs[k+'_er']<1.])
    
model_df=pd.DataFrame(model_vals)

model_df['spt']=[ teff_to_spt(x, sd=False) for x in models['teff']]
model_df['teff']= models['teff']
model_df=model_df.applymap(lambda x: float(x))
#model_df=pd.concat([model_line_df, model_df])

In [ ]:
#model_df= model_line_df

In [ ]:
#add will best's sample
#rename some columns
best_columns={}
for k in ['g', 'r', 'i', 'z', 'y']:
    best_columns.update({'absolute_PS1{}mag'.format(k):'PANSTARRS_{}'.format(k.upper())})
    best_columns.update({'absolute_PS1{}mag_er'.format(k):'PANSTARRS_{}_er'.format(k.upper())})
    best_columns.update({'use_mag_PS1{}mag'.format(k): 'use_PANSTARRS_{}'.format(k.upper())})
    

for k in ['J','H']:
    best_columns.update({'absolute_{}mag'.format(k): '2MASS {}'.format(k)})
    best_columns.update({'absolute_{}mag_er'.format(k): '2MASS {}_er'.format(k)})
    best_columns.update({'use_mag_{}mag'.format(k): 'use_2MASS {}'.format(k)})

#using Ks and K interchangably
for k in ['K']:
    best_columns.update({'absolute_{}smag'.format(k): '2MASS {}'.format(k)})
    best_columns.update({'absolute_{}smag_er'.format(k): '2MASS {}_er'.format(k)})
    best_columns.update({'use_mag_{}smag'.format(k): 'use_2MASS {}'.format(k)})

best_sample= pd.read_pickle('/Volumes/LaCie/Best2018_masked.csv.gz').rename(columns=best_columns)

#use the color relation to compute correction and glags
for k in dwarf_pairs.keys():
    #compute corr
    ref_k= get_correct_filter_name(dwarf_pairs[k]['ref'])
    
    corrk=   k+'and'+ref_k
    #pol= np.poly1d(FITS['colors_spt']['dwarfs'][corrk]['fit'])
    scatter=np.poly1d(FITS['colors_spt']['dwarfs'][corrk]['scatter'])
    
    pol= FITS['colors_spt']['dwarfs'][corrk]['fit']
    
    corrval= pol(best_sample.NumType.values)
    
    mags= best_sample[ref_k]+corrval
    unc= (best_sample[ref_k+'_er']**2+ scatter**2)**0.5
    
    best_sample[k]= mags
    best_sample[k+'_er']=unc
    best_sample['use_{}'.format(k)]=best_sample['use_{}'.format(ref_k)]

In [ ]:
best_sample['spt']=best_sample.NumType.values.astype(float)
best_sample['teff']=[ spt_to_teff(x, sd=False) for x in best_sample.spt.values]

In [ ]:
ABS_MAG_USED={}

In [ ]:
#visualize and fit 
for k in  dwarf_pairs.keys():
    fig, (ax, ax1)=plt.subplots(figsize=(12, 4), ncols=2)
    
    offset= np.nanmedian(df_dwarfs[k][df_dwarfs.teff.between(2500, 3000)])-np.nanmedian(model_df[k][model_df.teff.between(2500, 3000)])
   
    
    
    ax.errorbar(best_sample.spt, best_sample[k], yerr=best_sample[k+'_er'], fmt='o', alpha=0.1, label='Best et al')
    ax1.errorbar(best_sample.teff, best_sample[k], yerr=best_sample[k+'_er'], fmt='o', alpha=0.1, label='Best et al')
    
     
    ax.errorbar(model_df.spt, model_df[k]+offset, fmt='o', alpha=0.3, label='Models')
    ax1.errorbar(model_df.teff, model_df[k]+offset,  fmt='o', alpha=0.3, label='Models')
    
    ax.errorbar(df_dwarfs.spt, df_dwarfs[k],  fmt='o', alpha=0.1, label='Data')
    ax1.errorbar(df_dwarfs.teff, df_dwarfs[k], fmt='o', alpha=0.1, label='Data')
    
    
    #fits spectral types
    mask= ~np.logical_or.reduce([np.isnan(df_dwarfs.spt.values), 
                                np.isnan(df_dwarfs[k].values),
                                np.isnan(df_dwarfs[k+'_er'].values),
                                df_dwarfs[k+'_er'].values >1.])
    
    #mask models that are nans
    mask2=  ~np.logical_or.reduce([np.isnan(model_df.spt.values), 
                                np.isnan(model_df[k].values),
                                np.isnan(model_df[k+'_er'].values)])
    
    #mask best sample
    mask3=  ~np.logical_or.reduce([np.isnan(best_sample.spt.values), 
                                np.isnan(best_sample[k].values),
                                np.isnan(best_sample[k+'_er'].values),
                                   best_sample[k+'_er'].values >1.,
                                ~best_sample['use_{}'.format(k)].astype(bool)])
    
    
    #FITS
    x=np.concatenate([df_dwarfs.spt.values[mask],\
                      model_df.spt.values[mask2],\
                     best_sample.spt.values[mask3]])
    
    y= np.concatenate([df_dwarfs[k].values[mask], \
                       model_df[k].values[mask2]+offset,
                       best_sample[k].values[mask3]])
    
    yerr= np.concatenate([df_dwarfs[k+'_er'].values[mask], \
                          model_df[k+'_er'].values[mask2],\
                         best_sample[k+'_er'].values[mask3]])
    
    m, fit= fit_with_nsigma_clipping(x, y, yerr, 5, sigma=5)
    
    #fit=UnivariateSpline(x[np.argsort(x)], y[np.argsort(x)])
    
    
    #ax.scatter(x[m], y[m], s=20, color='k', marker='+')
    
    scatter=(np.nanmedian(np.abs(y[m]-fit(x[m])))**2+ np.nanmedian(yerr[m])**2)**0.5
    FITS['absmags_spt']['dwarfs'].update({k: {'x': 'spt',
                                          'y': k,
                                          'fit':fit,
                                          'scatter': scatter, \
                                          'method': 'spline',\
                                          'range':  (np.nanmin(x),\
                                                     np.nanmax(x)), \
                                         'x0':0}})
    
    ABS_MAG_USED.update({k: {'x': x[m],
                            'y':y[m],
                            'yerr':yerr[m],
                                                     }})
    

    xlim=(np.nanmin(x), np.nanmax(x))
    spgrid= np.arange(*xlim)
    ax.plot(spgrid, fit(spgrid), c='k')
    
    
    
    l=ax.legend()
    l.set_alpha(1)
    ax.set(title=k, xlabel='Spectral Type', ylabel='abs_{}'.format(k), ylim=[y[m].min(), y[m].max()])
    
    
    #fits for temperature
    
     
    #fits spectral types
    mask= ~np.logical_or.reduce([np.isnan(df_dwarfs.teff.values), 
                                 df_dwarfs.teff.values < 200,
                                np.isnan(df_dwarfs[k].values),
                                np.isnan(df_dwarfs[k+'_er'].values)])
    
    #mask models that are nans
    mask2=  ~np.logical_or.reduce([np.isnan(model_df.teff.values), 
                                   model_df.teff.values < 200,
                                np.isnan(model_df[k].values),
                                np.isnan(model_df[k+'_er'].values)])
    
    #mask best sample
    mask3=  ~np.logical_or.reduce([np.isnan(best_sample.teff.values), 
                                   best_sample.teff.values <200,
                                np.isnan(best_sample[k].values),
                                np.isnan(best_sample[k+'_er'].values),
                                ~ best_sample['use_{}'.format(k)].astype(bool)])
    
    
    #FITS
    xt=np.concatenate([df_dwarfs.teff.values[mask],\
                      model_df.teff.values[mask2],\
                     best_sample.teff.values[mask3]])
    
    yt= np.concatenate([df_dwarfs[k].values[mask], \
                       model_df[k].values[mask2]+offset,
                       best_sample[k].values[mask3]])
    
    yerrt= np.concatenate([df_dwarfs[k+'_er'].values[mask], \
                          model_df[k+'_er'].values[mask2],\
                         best_sample[k+'_er'].values[mask3]])
    
    m, fit= fit_with_nsigma_clipping(xt, yt, yerrt, 5, sigma=5)
    
   
    
    scatter= (np.nanmedian(np.abs(yt[m]-fit(xt[m])))**2+ np.nanmedian(yerrt[m])**2)**0.5
    
    FITS['absmags_teff']['dwarfs'].update({k: {'x': 'teff',
                                          'y': k,
                                          'fit':fit,\
                                          'scatter': scatter,
                                          'method': 'polynomial',\
                                          'range':  (np.nanmin(xt),\
                                                     np.nanmax(xt)), \
                                         'x0':0}})
    
    
    l=ax1.legend()
    l.set_alpha(1)
    teffgrid= np.arange(np.nanmin(xt), np.nanmax(xt))
    #ax1.scatter(xt[m], yt[m], s=20, color='k', marker='+')
    ax1.plot(teffgrid, fit(teffgrid), c='k')
    plt.tight_layout()
    
    ax1.set(title=k, xlabel='Teff', ylabel='abs_{}'.format(k), ylim=[yt[m].min(), yt[m].max()])
    plt.savefig('/users/caganze/figures/absmag_relations{}.jpeg'.format(k), bbox_inches='tight')

In [ ]:
FITS['absmags_spt']['dwarfs']['LSST_G']

In [ ]:
#make a teff magnitude relation 

In [ ]:
np.save('/Volumes/LaCie/popsimsdata/abs_mag_relations.npy', [FITS], allow_pickle=True) 
np.save('/users/caganze/research/popsimsdata/abs_mag_relations.npy', [FITS], allow_pickle=True)

In [ ]:
def bin_by_spt_bin(grid, xvals, yvals, yvalserr):
    #bin on a uniformly-spaced grid
    res=[]
    dx= grid[1]-grid[0]
    for x in grid:
        mask= np.logical_and(xvals>=x, xvals<x+dx)
        yx=np.nanmedian(yvals[mask])
        yxerr=(np.nanstd(yvals[mask])**2+np.nanmedian(yvalserr[mask])**2)**0.5
        
        res.append([yx, yxerr])
        
    return res
    

In [ ]:
bins=np.arange(10, 40)
df=pd.DataFrame()
df['spt']=[splat.typeToNum(x) for x in bins]
for k in ABS_MAG_USED.keys():
    x=ABS_MAG_USED[k]['x']
    y=ABS_MAG_USED[k]['y']
    yerr=ABS_MAG_USED[k]['yerr']
    vls= np.vstack(bin_by_spt_bin(bins, x, y, yerr))
    
    df[k]=['{:.1f}+/-{:.1f}'.format(x[0], x[1]) for x in vls]
df[['spt','LSST_G', 'LSST_R', 'LSST_I', 'LSST_Y', 'LSST_Z']].replace('nan+/-nan', '').to_latex('/data/lsst_dwarfs.tex', index=False)
df.replace('nan+/-nan', '')

In [ ]:
FITS['absmags_spt']['subdwarfs']